<a href="https://colab.research.google.com/github/mushrafi88/Julia-Set-Animation-using-Cuda-Acceleration/blob/master/julia_set_using_cuda_acceleration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numba import cuda

@cuda.jit(device=True)
def julia_kernel(z, c, max_iterations):
    for i in range(max_iterations):
        if abs(z) > 2:
            return i
        z = z**2 + c
    return max_iterations

@cuda.jit
def compute_julia_set_kernel(julia_set, c, max_iterations, width, height, zoom):
    x, y = cuda.grid(2)
    if x < width and y < height:
        zx = -2 * zoom + 4 * zoom * x / (width - 1)
        zy = -2 * zoom + 4 * zoom * y / (height - 1)
        z = complex(zx, zy)
        julia_set[y, x] = julia_kernel(z, c, max_iterations)

def compute_julia_set(c, max_iterations=1000, width=800, height=800, zoom=1):
    julia_set = np.zeros((height, width), dtype=np.uint16)
    threadsperblock = (16, 16)
    blockspergrid_x = int(np.ceil(width / threadsperblock[0]))
    blockspergrid_y = int(np.ceil(height / threadsperblock[1]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)
    compute_julia_set_kernel[blockspergrid, threadsperblock](julia_set, c, max_iterations, width, height, zoom)
    return julia_set

def plot_julia_set(c, filename, max_iterations=1000, width=800, height=800, zoom=1):
    julia_set = compute_julia_set(c, max_iterations, width, height, zoom)
    plt.figure(figsize=(width / 100, height / 100), dpi=100)
    plt.imshow(julia_set, cmap='viridis', extent=(-2 * zoom, 2 * zoom, -2 * zoom, 2 * zoom))
    plt.axis('off')
    plt.savefig(filename, bbox_inches='tight', pad_inches=0)
    plt.show()

a = 1.2
c = 0.7885 * np.exp(1j * a)
filename = "julia_set_a_{}.png".format(a)

plot_julia_set(c, filename)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numba import cuda
import os

@cuda.jit(device=True)
def julia_kernel(z, c, max_iterations):
    for i in range(max_iterations):
        if abs(z) > 2:
            return i
        z = z**2 + c
    return max_iterations

@cuda.jit
def compute_julia_set_kernel(julia_set, c, max_iterations, width, height, zoom):
    x, y = cuda.grid(2)
    if x < width and y < height:
        zx = -2 * zoom + 4 * zoom * x / (width - 1)
        zy = -2 * zoom + 4 * zoom * y / (height - 1)
        z = complex(zx, zy)
        julia_set[y, x] = julia_kernel(z, c, max_iterations)

def compute_julia_set(c, max_iterations=1000, width=800, height=800, zoom=1):
    julia_set = np.zeros((height, width), dtype=np.uint16)
    threadsperblock = (16, 16)
    blockspergrid_x = int(np.ceil(width / threadsperblock[0]))
    blockspergrid_y = int(np.ceil(height / threadsperblock[1]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)
    compute_julia_set_kernel[blockspergrid, threadsperblock](julia_set, c, max_iterations, width, height, zoom)
    return julia_set

def save_julia_set_image(c, filename, max_iterations=1000, width=800, height=800, zoom=1):
    julia_set = compute_julia_set(c, max_iterations, width, height, zoom)
    plt.figure(figsize=(width / 100, height / 100), dpi=100)
    plt.imshow(julia_set, cmap='viridis', extent=(-2 * zoom, 2 * zoom, -2 * zoom, 2 * zoom))
    plt.axis('off')
    plt.savefig(filename, bbox_inches='tight', pad_inches=0)
    plt.close()

def generate_julia_sequence(output_folder, num_images=1000):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    a_values = np.linspace(0, 2 * np.pi, num_images)
    for i, a in enumerate(a_values):
        c = 0.7885 * np.exp(1j * a)
        filename = os.path.join(output_folder, f"julia_set_a_{i}.png")
        save_julia_set_image(c, filename)
        print(f"Saved image {i+1}/{num_images}: {filename}")

output_folder = "julia_sequence"
generate_julia_sequence(output_folder)
